# *An Example With ScaPy*

ScaPy is a library that can be used to create and manipulate network packets, but it is also capable of doing minor attacks as well. There are going to be a few examples of what ScaPy can do.

- Creating basic packets
- tracerouting 
- attacks   



In [5]:
from scapy.all import *

# Define target IP and port (localhost and HTTP port)
target_ip = "127.0.0.1"
target_port = 80

# Craft the HTTP POST request with "Hello, World!" as the body
http_request = (
    b"POST / HTTP/1.1\r\n"
    b"Host: localhost\r\n"
    b"Content-Type: text/plain\r\n"
    b"Content-Length: 13\r\n\r\n"  # The length of "Hello, World!" is 13
    b"Hello, World!"
)

# Build the IP layer and TCP layer
ip_layer = IP(dst=target_ip)
tcp_layer = TCP(dport=target_port, sport=RandShort(), flags="S")

# Send SYN to initiate a TCP connection and receive SYN-ACK
syn_ack = sr1(ip_layer/tcp_layer, timeout=1)

# Complete the TCP handshake with ACK if SYN-ACK received
if syn_ack and syn_ack.haslayer(TCP) and syn_ack[TCP].flags == 0x12:
    # Craft ACK packet
    ack_packet = ip_layer / TCP(dport=target_port, sport=syn_ack[TCP].dport, seq=syn_ack.ack, ack=syn_ack.seq + 1, flags="A")
    send(ack_packet)

    # Send HTTP POST request with "Hello, World!" in the body
    send(ip_layer / TCP(dport=target_port, sport=syn_ack[TCP].dport, seq=syn_ack.ack, ack=syn_ack.seq + 1, flags="PA") / http_request)
    
    # Receive the server's response
    response = sr1(ip_layer / TCP(dport=target_port, sport=syn_ack[TCP].dport, seq=syn_ack.ack + len(http_request), ack=syn_ack.seq + 1, flags="A"), timeout=1)
    
    if response:
        response.show()
else:
    print("Failed to establish a TCP connection.")


Begin emission
.*.
Finished sending 1 packets

Received 3 packets, got 1 answers, remaining 0 packets
.
Sent 1 packets.
.
Sent 1 packets.
Begin emission
.*
Finished sending 1 packets

Received 2 packets, got 1 answers, remaining 0 packets
###[ IP ]###
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 40
  id        = 0
  flags     = DF
  frag      = 0
  ttl       = 64
  proto     = tcp
  chksum    = 0x3cce
  src       = 127.0.0.1
  dst       = 127.0.0.1
  \options   \
###[ TCP ]###
     sport     = http
     dport     = 62268
     seq       = 2485215397
     ack       = 0
     dataofs   = 5
     reserved  = 0
     flags     = R
     window    = 0
     chksum    = 0xc98a
     urgptr    = 0
     options   = []



Before executing the above code wireshark was opened and set to capture mode. This experiment to prevent clutter of wireshark interface only uses the loopback interface which is for local host. Below you can see the screenshot from the wireshark network trace from above if you look closely at the screenshot you can see a ***hello, world*** in the network trace 


![Alt Text](screenshots/Screenshot%20from%202024-11-05%2021-34-43.png)


# ***Sniffing packets using scapy***

So, I would like to test the general functinality of how ScaPy would do packet sniffing so I will be uisng my laptop to run this experiment, hopefully I do not get kicked off the web.....  

so first I will use IW to create a virtual interface and then I will use it to monitor the wifi traffic from my computer. 

so first thing is using the following command to set-up the virtual interface in wireless mode

*sudo iw dev wlp0s20f3 interface add mon0 type monitor*

and then using the following scapy mode to capture any data being received

**None of my traffic was being detected through the virtual interface so I decided to go with the standard interface on my machine**

In [7]:
from scapy.all import *

def packet_handler(packet):
    if packet.haslayer("DNS") and packet["DNS"].qr == 0:  # qr=0 indicates a DNS query
        domain_name = packet["DNS"].qd.qname.decode()
        print(f"DNS Query for: {domain_name}")

# Start sniffing on the interface
sniff(iface="wlp0s20f3", prn=packet_handler, filter="udp port 53", store=0)  # Capture DNS on port 53

DNS Query for: mobile.events.data.microsoft.com.
DNS Query for: mobile.events.data.microsoft.com.
DNS Query for: onedscolprdwus20.westus.cloudapp.azure.com.
DNS Query for: ipv4only.arpa.
DNS Query for: mozilla.cloudflare-dns.com.
DNS Query for: mozilla.cloudflare-dns.com.
DNS Query for: my-haproxy.umassd.edu.
DNS Query for: login_umassd_edu.alias.cirrusidentity.com.
DNS Query for: doh.test.
DNS Query for: doh.test.
DNS Query for: ipv4only.arpa.
DNS Query for: connectivity-check.ubuntu.com.
DNS Query for: notifications.netflix.com.
DNS Query for: notifications.netflix.com.
DNS Query for: aax-us-east-retail-rtb.amazon.com.
DNS Query for: aax-us-east-retail-rtb.amazon.com.
DNS Query for: aax-us-east-retail-rtb.amazon.com.
DNS Query for: aax-us-east-retail-rtb.amazon.com.
DNS Query for: aax-us-east-retail-rtb.amazon.com.umassd.edu.
DNS Query for: aax-us-east-retail-rtb.amazon.com.umassd.edu.
DNS Query for: aax-us-east-retail-rtb.amazon.com.umassd.edu.
DNS Query for: aax-us-east-retail-rtb.

<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>

For reference the only webpages I opened up are netflix, hulu, and amazon, respectively and hulu did not appear in the list of the websties that was found. but when amazon was opened up there was a lot of traffic that came through.

#Reading Pcaps


Scapy can be used to read PCAPS so now we will be using the the pcap from earlier to see how scapy can read and then process it into a pdf

In [11]:
from scapy.all import rdpcap
import pyx
global network_trace 

network_trace = rdpcap('network_traffic_files/test1.pcap')

print(network_trace)

<test1.pcap: TCP:19 UDP:12 ICMP:0 Other:0>


Now we can go through and send all of the packet information to a pdf

In [12]:
network_trace[22].show()

###[ Ethernet ]###
  dst       = 00:00:00:00:00:00
  src       = 00:00:00:00:00:00
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 139
     id        = 24377
     flags     = DF
     frag      = 0
     ttl       = 1
     proto     = udp
     chksum    = 0x1bf3
     src       = 127.0.0.53
     dst       = 127.0.0.1
     \options   \
###[ UDP ]###
        sport     = domain
        dport     = 54083
        len       = 119
        chksum    = 0xfebe
###[ DNS ]###
           id        = 32758
           qr        = 1
           opcode    = QUERY
           aa        = 0
           tc        = 0
           rd        = 1
           ra        = 1
           z         = 0
           ad        = 0
           cd        = 0
           rcode     = ok
           qdcount   = 1
           ancount   = 2
           nscount   = 0
           arcount   = 1
           \qd        \
            |###[ DNS Question Record ]###
            |  qname   

In [7]:
import pyx.canvas
from scapy.all import *
import pyx

packet_cavnas = pyx.canvas.canvas()

network_trace = rdpcap('network_traffic_files/test1.pcap')
network_trace[15].pdfdump(filename="reports/packet15.pdf", layer_shift=1)

ImportError: PyX and its dependencies must be installed